In [6]:
"""Import all the relevant libraries."""

from mkd_blockchain import MKDBlockchain
from blockchain_utils import BlockchainUtils
import block
from pubsub import pub
from node import Node
from transaction import Transaction
from transaction_pool import TransactionPool
from wallet import Wallet
from Crypto.PublicKey import RSA
import pprint
import random
import time
from mkdtree import kdtree

In [10]:
"""Initialize Nodes with Genesis Block"""

print('START')
MAX_NODES = 9999
num_nodes = 80

node_id_list = [int(MAX_NODES / 2)]
mkd_blkchain = MKDBlockchain()

nodes = [] #list of all nodes

#itterate through and initialize each node with the genesis block
for i in range(num_nodes):
    new_node = Node(i+1, 0)
    new_node.blockchain = mkd_blkchain
    nodes.append(new_node)

print(nodes)


START
[<node.Node object at 0x000002130954B910>, <node.Node object at 0x0000021306E438E0>, <node.Node object at 0x0000021306E3DD90>, <node.Node object at 0x000002130953D820>, <node.Node object at 0x000002130760FF10>, <node.Node object at 0x00000213079E0550>, <node.Node object at 0x000002130962D2E0>, <node.Node object at 0x0000021306E480D0>, <node.Node object at 0x0000021306E481F0>, <node.Node object at 0x0000021309636970>, <node.Node object at 0x000002130954BD90>, <node.Node object at 0x0000021309636E80>, <node.Node object at 0x000002130954B4C0>, <node.Node object at 0x000002130954B790>, <node.Node object at 0x0000021309636760>, <node.Node object at 0x0000021309638160>, <node.Node object at 0x00000213096369A0>, <node.Node object at 0x0000021307625730>, <node.Node object at 0x0000021309638BE0>, <node.Node object at 0x0000021309639A00>, <node.Node object at 0x0000021309638640>, <node.Node object at 0x0000021309638610>, <node.Node object at 0x0000021309639FD0>, <node.Node object at 0x0000

In [17]:
sample = nodes[1]
sample2 = nodes[2]
print(sample.blockchain.blocks)
print(sample2.blockchain.blocks)

<KDNode - Block([40, 0, 0, 1625679474.351727], [])>
<KDNode - Block([40, 0, 0, 1625679474.351727], [])>


'\n#Opens the data set file and generates nodes based on the provided data.\n#file is comma delimited in the following format: time-nodeID-xcoor-ycoor-miner-region-prevregion\nwith open(\'node_data.txt\') as f:\n    lines = f.readlines() # list containing lines of file\n    i = 0\n    node = []\n    for line in lines:\n        parts = line.split(\',\')\n        #itterates through based on the number of nodes provided.\n        if i < num_nodes:\n            new_node = Node(i + 1, parts[5])\n            new_node.startListener("c" + str(parts[5]).strip())\n            node.append(new_node)\n        else:\n            move_listener(node[i % num_nodes].node_listener, "c" + str(parts[6]).strip(), "c" + str(parts[5]).strip())\n        i += 1\n\nprint(tree)\nprint(tree.sel_axis)\n'